In [1]:
import torch.nn as nn 
import torch 
import torchvision 
import cv2 
import numpy as np 
import matplotlib.pyplot as plt 
from PIL import Image
from tqdm import tqdm 
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader,Dataset
import os 
import yaml 
import wandb  
import pickle 

from src import Convolution_Auto_Encoder, Mnist_Dataset,MVtecADDataset,Datadir_init
from src import MVtecEncoder,MVtecDecoder,Convolution_Auto_Encoder
from src import Machine_Metric,Reconstruction_Metric

cfg = {'seed':42,
       'img_size':256,
       'device':'cuda:0',
       'encoded_space_dim':256,
       'lr':0.001,
       'weight_decay':1e-05,
       'batch_size':32,
       'Epochs':50,
       'target_class':6,
       'save_dir':'MVtecAD3',
       'Dataset_dir':'./Dataset/hazelnut',
       'optimizer':'adamw',
       'Decription':'Normalize제외 하고 진행'}

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def preprocess(cfg,augmentation=None):
    #mk save dir 
    try:
        os.mkdir(f"./Save_models/{cfg['save_dir']}")
    except:
        pass
    torch.manual_seed(cfg['seed'])
    data_dir = cfg['Dataset_dir']
    Data_dir = Datadir_init()
    train_dirs = Data_dir.train_load()
    test_dirs,test_labels = Data_dir.test_load()
    indx = int(len(train_dirs)*0.8)

    train_dset = MVtecADDataset(cfg,train_dirs[:indx],Augmentation=augmentation)
    valid_dset = MVtecADDataset(cfg,train_dirs[indx:],Augmentation=augmentation)
    test_dset = MVtecADDataset(cfg,test_dirs,test_labels,Augmentation=augmentation)

    train_loader = DataLoader(train_dset,batch_size=cfg['batch_size'],shuffle=True)
    valid_loader = DataLoader(valid_dset,batch_size=cfg['batch_size'],shuffle=False)
    test_loader = DataLoader(test_dset,batch_size=cfg['batch_size'],shuffle=False)
    return train_loader,valid_loader,test_loader 

def train_epoch(model,dataloader,criterion,optimizer,scheduler,scaler):
       model.train()
       optimizer.zero_grad()
       train_loss = [] 
       for img,label in dataloader:
              img = img.to(cfg['device']).type(torch.float32)
              with torch.cuda.amp.autocast():
                    y_pred = model(img).type(torch.float32)
                    loss = torch.sqrt(criterion(img,y_pred))
              #y_pred = model(img).type(torch.float32)
              

              #Backpropagation
              scaler.scale(loss).backward()
              scaler.step(optimizer)
              scaler.update() 
              #loss.backward()
              #optimizer.step()

              #loss save 
              train_loss.append(loss.detach().cpu().numpy())
       scheduler.step() 
       print(f'\t epoch : {epoch+1} train loss : {np.mean(train_loss):.3f}')
       return np.mean(train_loss)

def valid_epoch(model,dataloader,criterion):
       model.eval()
       valid_loss = [] 
       with torch.no_grad():
              for img,label in dataloader:
                     img = img.to(cfg['device'])
                     y_pred = model(img)
                     loss = torch.sqrt(criterion(y_pred,img))
                     valid_loss.append(loss.detach().cpu().numpy())
       print(f'\t epoch : {epoch+1} valid loss : {np.mean(valid_loss):.3f}')
       fig, (ax1,ax2) = plt.subplots(ncols=2,nrows=1,figsize=(5, 5))
       ax1.imshow(img[0].detach().cpu().permute(1,2,0).numpy())
       ax2.imshow(y_pred[0].detach().cpu().permute(1,2,0).numpy())
       plt.show()
       return np.mean(valid_loss)   

def Save_resuld(cfg):
    metric =  {} 
    metric['auto'] = {} 
    metric['machine']={}

    cfg = yaml.load(open('./Save_models/MVtecAD4/config.yaml','r'), Loader=yaml.FullLoader)
    machine = Machine_Metric(cfg)
    auto = Reconstruction_Metric(cfg)

    acc,pre,rec,f1 = machine.main()
    [AUROC,ROC], [ACC,PRE,RECALL,F1] = auto.main() 

    metric['auto']['auroc'] = AUROC
    metric['auto']['roc'] = ROC 
    metric['auto']['metric'] =[ACC,PRE,RECALL,F1]
    metric['machine']['metric']=[acc,pre,rec,f1]

    with open(f"./Save_models/{cfg['save_dir']}/temp.pickle",'wb') as fw:
        pickle.dump(metric, fw)       

In [ ]:
#train_loader, valid_normal_loader,valid_anomal_loader  = preprocess(cfg)
train_loader,valid_loader,test_loader   = preprocess(cfg)
model = Convolution_Auto_Encoder(MVtecEncoder,MVtecDecoder,cfg['encoded_space_dim']).to(cfg['device'])
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=cfg['lr'],weight_decay=cfg['weight_decay'])

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=100,eta_min=0)
scaler = torch.cuda.amp.GradScaler()


total_train_loss = [] 
total_valid_loss = [] 
best_valid_loss = np.inf 

for epoch in tqdm(range(cfg['Epochs'])):
    train_loss = train_epoch(model,train_loader,criterion,optimizer,scheduler,scaler)
    valid_loss = valid_epoch(model,valid_loader,criterion)
    total_train_loss.append(train_loss)
    total_valid_loss.append(valid_loss)

    if valid_loss < best_valid_loss:
        torch.save(model,f"./Save_models/{cfg['save_dir']}/best.pt")
        best_valid_loss = valid_loss 
        print(f'\t Model save : {epoch} | best loss : {best_valid_loss :.3f}')
        cfg['save_epoch'] = epoch 
        cfg['best_valid_loss'] = best_valid_loss
    
    if valid_loss != valid_loss:
        model = torch.load(f"./Save_models/{cfg['save_dir']}/best.pt")
        print('Model rewinded')
cfg['last_train_loss'] = total_train_loss[-1]
cfg['last_valid_loss'] = total_valid_loss[-1]

f = open(f"./Save_models/{cfg['save_dir']}/config.yaml",'w+')
yaml.dump(cfg, f, allow_unicode=True)
    
    